In [ ]:
import inspect

import matplotlib.pyplot as plt
import numpy as np

# L96 two-timescale model description





In 1996, Lorenz proposed a two-time scale dynamical system, referred to as the Lorenz-96 model (L96), whose behavior mimics the non-linear dynamics of the extratropical atmosphere with simplified representation of multiscale interactions and nonlinear advection [Lorenz (1996)](https://www.ecmwf.int/en/elibrary/10829-predictability-problem-partly-solved). The L96 model consists of a set of equations coupling two scales of variables evolving over slow and fast timescales and arranged around a latitude circle as shown in Fig. 1.
[Lorenz (1996)](https://www.ecmwf.int/en/elibrary/10829-predictability-problem-partly-solved) describes the "two time-scale" model in a set of equations (1 and 2) given by:
\begin{align}
\frac{d}{dt} X_k
&= - X_{k-1} \left( X_{k-2} - X_{k+1} \right) - X_k + F - \left( \frac{hc}{b} \right) \sum_{j=0}^{J-1} Y_{j,k}
\\
\frac{d}{dt} Y_{j,k}
&= - cbY_{j+1,k} \left( Y_{j+2,k} - Y_{j-1,k} \right) - c Y_{j,k} + \frac{hc}{b} X_k
\end{align}

The model includes $K$ slow (or low-frequency) variables, denoted as $X_k$, $k=1,\ldots,K$ and each slow variable $X_k$ is coupled with $J$ fast (or high-frequency) variables $Y_{j,k}$ , $j=1,\ldots,J$. 
The model equations are coupled via the mean term $\sum_{j=0}^{J-1} Y_{j,k}$, and the coupling depends on three key parameters: $b$, $c$ and $h$. The parameter $b$ determines the magnitude of the nonlinear interactions among the fast variables. The parameter $c$ controls how rapidly the fast variables fluctuate compared to the slow variables. Finally, the parameter $h$ governs the strength of the coupling between the slow and fast variables. 

The chaotic dynamical system L96 is very useful for testing different numerical methods in atmospheric modeling thanks to its transparency, low computational cost and simplicity compared to Global Climate Models (GCM). The interaction between variables of different scales makes the L96 model of particular interest when evaluating new parameterization methodologies. As such, it was used in assessing different techniques that were later incorporated into GCMs ([Crommelin (2008)](https://journals.ametsoc.org/view/journals/atsc/65/8/2008jas2566.1.xml), [Dorrestijn (2013)](https://royalsocietypublishing.org/doi/10.1098/rsta.2012.0374)).

The L96 model has been extensively used as a test bed in various studies including data assimilation approaches ([Law (2016)](https://www.sciencedirect.com/science/article/pii/S0167278915002766?via%3Dihub), [Hatfield (2018)](https://journals.ametsoc.org/view/journals/mwre/146/1/mwr-d-17-0132.1.xml)), stochastic parameterization schemes ([Kwasniok (2012)](https://royalsocietypublishing.org/doi/10.1098/rsta.2011.0384), [Arnold (2013)](https://royalsocietypublishing.org/doi/full/10.1098/rsta.2011.0479), [Chorin (2015)](https://www.pnas.org/doi/10.1073/pnas.1512080112))  and Machine Learning-based parameterization methodologies ([Schneider (2017)](https://agupubs.onlinelibrary.wiley.com/doi/10.1002/2017GL076101), [Dueben (2018)](https://gmd.copernicus.org/articles/11/3999/2018/), [Watson (2019)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2018MS001597), [Gagne (2020)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2019MS001896)).
Finally, note that the $F$ term in equation (1) is missing in the original Lorenz 96 paper but the author provides the parameter. All other authors use the term as well, e.g. [Wilks (2005)](http://rmets.onlinelibrary.wiley.com/doi/abs/10.1256/qj.04.03).)


<img src="https://www.researchgate.net/publication/319201436/figure/fig1/AS:869115023589376@1584224577926/Visualisation-of-a-two-scale-Lorenz-96-system-with-J-8-and-K-6-Global-scale-values.png" width=400> *Fig. 1: Visualisation of a two-scale Lorenz '96 system with J = 8 and K = 6. Global-scale variables ($X_k$) are updated based on neighbouring variables and on the local-scale variables ($Y_{j,k}$) associated with the corresponding global-scale variable. Local-scale variabless are updated based on neighbouring variables and the associated global-scale variable. The neighbourhood topology of both local and global-scale variables is circular. Image from [Exploiting the chaotic behaviour of atmospheric models with reconfigurable architectures - Scientific Figure on ResearchGate.](https://www.researchgate.net/figure/Visualisation-of-a-two-scale-Lorenz-96-system-with-J-8-and-K-6-Global-scale-values_fig1_319201436)*



In [ ]:
from L96_model import L96_2t_xdot_ydot

print(inspect.getsource(L96_2t_xdot_ydot))

In [ ]:
from L96_model import integrate_L96_2t

print(inspect.getsource(integrate_L96_2t))

In [ ]:
J = 10  # Number of local-scale Y variables per single global-scale X variable
K = 36  # Number of globa-scale variables X
nt = 1000  # Number of time steps
si = 0.005  # Sampling time interval
dt = 0.005  # Time step
F = 10.0  # Focring
h = 1.0  # Coupling coefficient
b = 10.0  # ratio of amplitudes
c = 10.0  # time-scale ratio


def s(k, K):
    """A non-dimension coordinate from -1..+1 corresponding to k=0..K"""
    return 2 * (0.5 + k) / K - 1


k = np.arange(K)  # For coordinate in plots
j = np.arange(J * K)  # For coordinate in plots
# Initial conditions
Xinit = s(k, K) * (s(k, K) - 1) * (s(k, K) + 1)
Yinit = 0 * s(j, J * K) * (s(j, J * K) - 1) * (s(j, J * K) + 1)

# "Run" model
X, Y, t = integrate_L96_2t(Xinit, Yinit, si, nt, F, h, b, c, dt=dt)

plt.figure(figsize=(12, 10))
plt.subplot(221)
# Snapshot of X[k]
plt.plot(k, X[-1], label="$X_k(t=n_t dt)$")
plt.plot(j / J, Y[-1], label="$Y_{j,k}(t=n_t dt)$")
plt.plot(k, Xinit, "k:", label="$X_k(t=0)$")
plt.plot(j / J, Yinit, "k:", label="$Y_{j,k}(t=0)$")
plt.legend()
plt.xlabel("k, k+j/J")
plt.title("$X_k, Y_{j,k}$")
plt.subplot(222)
# Sample time-series X[0](t), Y[0](t)
plt.plot(t, X[:, 0], label="$X_0(t)$")
plt.plot(t, Y[:, 0], label="$Y_{0,0}(t)$")
plt.legend()
plt.xlabel("t")
plt.subplot(223)
# Full model history of X
plt.contourf(k, t, X)
plt.colorbar(orientation="horizontal")
plt.xlabel("k")
plt.ylabel("t")
plt.title("$X_k(t)$")
plt.subplot(224)
# Full model history of Y
plt.contourf(j / J, t, Y)
plt.colorbar(orientation="horizontal")
plt.xlabel("k+j/J")
plt.ylabel("t")
plt.title("$Y_{j,k}(t)$");

In [ ]:
from L96_model import L96

help(L96)

In [ ]:
# Create an instance of the L96 model, default parameters except those given
M = L96(36, 10, F=10, dt=0.005)
M

In [ ]:
# Set the initial conditions (here X is the same cubic as above, Y=0)
M.set_state(s(M.k, M.K) * (s(M.k, M.K) - 1) * (s(M.k, M.K) + 1), 0 * M.j)
print(M)

In [ ]:
# Run the model for 1000 sample intervals, or for time 1000*0.005 = 5
X2, Y2, t = M.run(0.005, 5)

print("Mean absolute difference =", np.abs(X2 - X).mean() + np.abs(Y2 - Y).mean())

In [ ]:
# The one line version of the above
X3, Y3, t = (
    L96(36, 10, F=10, dt=0.005)
    .set_state(s(M.k, M.K) * (s(M.k, M.K) - 1) * (s(M.k, M.K) + 1), 0 * M.j)
    .run(0.005, 5)
)

print("Mean absolute difference =", np.abs(X3 - X).mean() + np.abs(Y3 - Y).mean())